# G2Config Reference

## Prepare Environment

In [ ]:
import com.senzing.g2.engine.G2Config;
import com.senzing.g2.engine.G2ConfigJNI;

### Helper class for Json Rendering

In [ ]:
%%loadFromPOM
<dependency>
    <groupId>org.glassfish</groupId>
    <artifactId>javax.json</artifactId>
    <version>1.1.4</version>
</dependency>

In [ ]:
import javax.json.*;
import static java.util.Collections.*;
import static javax.json.stream.JsonGenerator.PRETTY_PRINTING;

In [ ]:
public class JsonUtil {
    private static final JsonWriterFactory PRETTY_FACTORY
        = Json.createWriterFactory(singletonMap(PRETTY_PRINTING, true));
        
    private static final JsonWriterFactory UGLY_FACTORY
        = Json.createWriterFactory(emptyMap());
    
    public static String toJsonText(JsonValue val) {
        return toJsonText(val, true);
    }

    public static String toJsonText(JsonValue val, boolean prettyPrint) {
        JsonWriterFactory factory = (prettyPrint) ? PRETTY_FACTORY : UGLY_FACTORY;
        StringWriter sw = new StringWriter();
        JsonWriter writer = factory.createWriter(sw);
        writer.write(val);
        sw.flush();
        return sw.toString();
    }
    
    public static JsonObject parseJsonObject(String jsonText) {
        if (jsonText == null) return null;
        StringReader sr = new StringReader(jsonText);
        JsonReader jsonReader = Json.createReader(sr);
        return jsonReader.readObject();
  }

    public static JsonArray parseJsonArray(String jsonText) {
        if (jsonText == null) return null;
        StringReader sr = new StringReader(jsonText);
        JsonReader jsonReader = Json.createReader(sr);
        return jsonReader.readArray();
  }
    
    
}

In [ ]:
import java.util.UUID;
String str;
public static void RenderJSON(Object obj){
    str = obj.toString();
    JsonObject json = JsonUtil.parseJsonObject(str);
    String Config = JsonUtil.toJsonText(json, false);
    UUID id = UUID.randomUUID();
    String uuid = id.toString();
    String div = "<div id=\""+ uuid +"\" style=\"height:100%; width:100%; background-color: LightCyan\"></div>";
    display(div, "text/html");
    String jav = "require([\"https://rawgit.com/caldwell/renderjson/master/renderjson.js\"], function() {document.getElementById(\'"+ uuid +"\').appendChild(renderjson("+json+"))});";
    display(jav, "application/javascript");
}

### Initialize Senzing configuration

Using environment variables and default values, create `senzingConfigJson`.
This value is used when instantiating Senzing objects.

In [ ]:
// Get variables used in constructing Senzing Engine configuration.
String senzingConfigJson=System.getenv("SENZING_ENGINE_CONFIGURATION_JSON");

String configPath = System.getenv("SENZING_ETC_DIR");
if (configPath == null) {
    configPath = "/etc/opt/senzing";
}

String supportPath = System.getenv("SENZING_DATA_VERSION_DIR");
if (supportPath == null) {
    supportPath = "/opt/senzing/data";
}

String g2Path = System.getenv("SENZING_G2_DIR");
if (g2Path == null) {
    g2Path = "/opt/senzing/g2";
}

String resourcePath = g2Path + "/resources";

String sqlConnection = System.getenv("SENZING_SQL_CONNECTION");
if (sqlConnection == null) {
    sqlConnection = "sqlite3://na:na@/var/opt/senzing/sqlite/G2C.db";
}

    // Construct the JSON string used for Senzing Engine configuration.
if(senzingConfigJson==null)
    senzingConfigJson = "{"
       + "\"PIPELINE\": {"
       +     "\"CONFIGPATH\": \"" + configPath + "\","
       +     "\"SUPPORTPATH\": \"" + supportPath + "\","
       +     "\"RESOURCEPATH\": \"" + resourcePath + "\""
       + "},"
       + "\"SQL\": {"
       +     "\"CONNECTION\": \"" + sqlConnection + "\""
       + "}}";

RenderJSON(senzingConfigJson);

## G2Config

The G2Config API is used to create a new JSON configuration from the default template
or manipulate a JSON configuration that it is provided.
The API read/writes a JSON string, but that string would typically be
read/written from/to a g2config.json file that the G2 engine reads during the initV2(...) call.

### G2Config Initialization

To start using G2Config, you must first create and initialize the G2Config object.
This should be done once per process.

Create a new instance of the Senzing G2Config object and assign it to a variable.
Then, call the appropriate initialization method (such as initV2) to initialize the G2Config object.

During the call, the initialization methods accept the following parameters:

- **moduleName:** A short name given to this instance of the G2 engine (i.e. your G2Module object)
- **senzingConfigJson:** A JSON document containing system parameters (see the section called "Initialization Parameters")
- **verboseLogging:** A boolean which enables diagnostic logging - this will print a massive amount of information to stdout (default = False)

Calling these functions will return "0" upon success - useful for error handling.

In [ ]:
String moduleName = "ExampleG2Engine";
boolean verboseLogging = true;

G2Config g2config = new G2ConfigJNI();
int return_code = g2config.initV2(moduleName, senzingConfigJson, verboseLogging);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    System.out.print(return_code);

In [ ]:
long config_handle = g2config.create()

### List DataSources
Call G2Config's `listDataSourcesV2()` method and print results.

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2config.listDataSourcesV2(config_handle, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    RenderJSON(response);

### Add DataSource
Call G2Config's `addDataSourceV2()` method and print results

In [ ]:
String dataSource = "{\"DSRC_CODE\": \"CUSTOMER\"}";

StringBuffer response = new StringBuffer();

int return_code = g2config.addDataSourceV2(config_handle, dataSource, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    RenderJSON(response);

### List DataSources again
Call G2Config's `listDataSourcesV2()` method and print results. Notice that the list now contains the newly add dataSource of "CUSTOMER".

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2config.listDataSourcesV2(config_handle, response);

if(return_code!=0)
    System.out.print(g2config.getLastException());
else
    RenderJSON(response);

## Close configuration handle

In [ ]:
g2config.close(config_handle);